In [64]:
import numpy as np
from scipy import stats
from scipy.stats import entropy
from pywt import wavedec
from scipy.signal import welch


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mne_features.univariate import compute_spect_entropy, compute_wavelet_coef_energy
# https://mne.tools/mne-features/api.html

import mne
from mne.io import concatenate_raws, read_raw_edf
import glob


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP, SPoC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mne.decoding import (
    SlidingEstimator,
    GeneralizingEstimator,
    Scaler,
    cross_val_multiscore,
    LinearModel,
    get_coef,
    Vectorizer,
    CSP,
)
import numpy as np
from mne.preprocessing import ICA

from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from utils import preprocess_data


import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA

https://arxiv.org/ftp/arxiv/papers/1312/1312.2877.pdf






After the AAR process, the continuous EEG data were
epoched by extracting data epochs that are time locked to
specific event types.
When no sensory inputs or motor outputs are being
processed, the mu (8–12 Hz) and beta (13–30 Hz) rhythms are
said to be synchronized [4, 27]. These rhythms are
electrophysiological features that are associated with the
brain’s normal motor output channels [4, 27]. While preparing
for a movement or executing a movement, a desynchronization
of the mu and beta rhythms occurs which is referred to as ERD
and it can be extracted 1-2 seconds before onset of movement
(as depicted in Fig. 4). Later, these rhythms synchronize again
within 1-2 seconds after movement, and this is referred to as
ERS.
On the other hand, delta rhythms can be extracted from the
motor cortex, within the pre-movement stage, and this is
referred to MRCP. The slow (less than 3 Hz) MRCP is
associated with an event-related negativity that occurs 1-2
seconds before the onset of movement [28, 29].
In our experiments, we extracted time-locking events with
type = 3 (left hand) or type = 4 (right hand) with different
epoch limits and types of analysis:
- ERD analysis: epoch limits from -2 to 0 seconds.
- ERS analysis: epoch limits from 4.1 to 5.1 seconds.
- MRCP analysis: epoch limits from -2 to 0 seconds.

Independent Component Analysis (ICA)
After the AAR process, ICA was used to parse the
underlying electrocortical sources from EEG signals that are
affected by artifacts [30, 31]. Data decomposition using ICA
changes the basis linearly from data that are collected at single
scalp channels to a spatially transformed virtual channel basis.
Each row of the EEG data in the original scalp channel data
represents the time course of accumulated differences between
source projections to a single data channel and one or more
reference channels [32].
EEGLAB was used to run ICA on the described epoched
datasets (left and right ERD, ERS, and MRCP) for the channels
FC3, FCZ, FC4, C3, C1, CZ, C2, and C4.
F. Rhythm Isolation
A short IIR band pass filter from 8 to 30 Hz was applied on
the ERD/ERS epoched datasets of the experiment for the
purpose of isolating mu/beta rhythms. Another short IIR
lowpass filter of 3 Hz was applied on MRCP epoched datasets
for isolating delta rhythms. The result of this was 6 files for
each run: ERD/ERS and MRCP for both left and right hand
movements for each subject.
V. PRACTICAL IMPLEMENTATION AND RESULTS
A. Feature Vectors Construction and Numerical
Representation
After the EEG datasets were analyzed as described in the
previous section, the activation vectors were calculated for
each of the resulted epochs’ datasets as the multiplication of the
ICA weights and ICA sphere for each dataset subtracting the
mean of the raw data from the multiplication results.
Then, the mean, power, and energy of the activations were
calculated to construct the feature vectors. For each subject’s
single run, 6 feature vectors were extracted as <Power (8
features), Mean (8 features), Energy (8 features), Type (1
feature: ERS/ERD/MRCP), Side (1 target: Left/Right)>
resulting in a 10826 feature matrix.
The constructed features were represented in a numerical
format that is suitable for use with machine learning algorithms
[33, 34]. Every column in the features matrices was normalized
between 0.1 and 0.9 such that the datasets could be inputted to
the learning algorithms described in the next subsection.

In [65]:
files = glob.glob('../files/S001/*.edf')

'''
https://github.com/mne-tools/mne-python/blob/main/mne/datasets/eegbci/eegbci.py#L110
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws = []
f = [5,9,13]
# ,6,10,14]
for i in f:
    print(i)
    current_file = files[i-1]
    r = read_raw_edf(current_file, preload=True, stim_channel='auto')
    events, _ = mne.events_from_annotations(r)
    if i in [5, 9, 13]:
        new_labels_events = {1:'rest', 2:'action_hand', 3:'action_feet'} # action
    else:
        new_labels_events = {1:'rest', 2:'imagine_hand', 3:'imagine_feet'} # imagine
    new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
    r.set_annotations(new_annot)
    raws.append(r)
    
raw_obj = concatenate_raws(raws)

5
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
9
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
13
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['T0']


In [81]:
original_raw = raw_obj.copy()

# filters
notch_freq = 50
original_raw.notch_filter(notch_freq, fir_design='firwin')

low_cutoff = 0.5
high_cutoff = 79
original_raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

events, event_dict = mne.events_from_annotations(original_raw)
print(original_raw.info)
print(event_dict)
picks = mne.pick_types(original_raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')
eegbci.standardize(original_raw)
montage = mne.channels.make_standard_montage('standard_1005')
original_raw.set_montage(montage)

## ICA
n_components = 10
ica = ICA(n_components=n_components, random_state=97, max_iter=800)
ica.fit(original_raw)
components_to_excludes, scores = ica.find_bads_eog(original_raw, ch_name='Fpz')
if components_to_excludes is not None and len(components_to_excludes) > 0:
    ica.exclude = components_to_excludes
    original_raw = ica.apply(original_raw)
else:
    print("No components to exclude")

event_id = {'action_hand': 1, 'action_feet': 2}
events, event_dict = mne.events_from_annotations(original_raw, event_id=event_id)
tmin = -2  # Time before event in seconds
tmax = 4  # Time after event in seconds
epochs = mne.Epochs(original_raw, events, event_dict, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)


# ERD analysis
erd_epochs = mne.Epochs(original_raw, events, event_id=event_id, tmin=-2, tmax=0, baseline=None)

# ERS analysis
ers_epochs = mne.Epochs(original_raw, events, event_id=event_id, tmin=4.1, tmax=5.1, baseline=None)

# MRCP analysis
mrcp_epochs = mne.Epochs(original_raw, events, event_id=event_id, tmin=-2, tmax=0, baseline=None)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)


- Filter length: 1057 samples (6.606 sec)

Filtering raw data in 3 contiguous segments


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.2s finished


Setting up band-pass filter from 0.5 - 79 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 79.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 79.50 Hz)
- Filter length: 1057 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.2s finished


Used Annotations descriptions: ['action_feet', 'action_hand', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 79.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'action_feet': 1, 'action_hand': 2, 'rest': 3}
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 10 components
Fitting ICA took 0.6s.
Using EOG channel: Fpz
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequen

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [96]:
from mne.time_frequency import psd_multitaper

spectrum = erd_epochs.compute_psd(method="multitaper", fmin=8, fmax=30)


Using data from preloaded Raw for 30 events and 321 original time points ...
    Using multitaper spectrum estimation with 7 DPSS windows


In [98]:
spectrum.get_data().shape

(30, 64, 44)

In [104]:
import pywt

# Define the wavelet decomposition parameters
wavelet = 'db10'  # Choose a wavelet (e.g., 'morl')
level = 15  # Adjust the decomposition level as needed

# Perform wavelet decomposition on EEG data
coeffs = pywt.wavedec(spectrum.get_data(), wavelet, level=level)
mu_rhythm = coeffs[level][0] # Mu rhythm (8-13 Hz)
beta_rhythm = coeffs[level][1]  # Beta rhythm (13-30 Hz)

/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 15 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [118]:
mean_mu = np.mean(mu_rhythm, axis=1)
mean_beta = np.mean(beta_rhythm, axis=1)
print(mean_mu.shape)

(64,)


In [82]:
n_components = 10
ica = ICA(n_components=n_components, random_state=97, max_iter=800)
ica.fit(mrcp_epochs)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Using data from preloaded Raw for 30 events and 321 original time points ...
0 bad epochs dropped
Selecting by number: 10 components
Using data from preloaded Raw for 30 events and 321 original time points ...
Fitting ICA took 0.2s.


Method,fastica
Fit,14 iterations on epochs (9630 samples)
ICA components,10
Available PCA components,64
Channel types,eeg
ICA components marked for exclusion,—


In [87]:
ica_weights = ica.get_components()
whitening_matrix = ica.get_components()

# Calculate the whitening and sphering transformation matrix
ica_sphere = np.linalg.pinv(np.dot(whitening_matrix, ica_weights.T))

In [178]:
feature_matrix = []
ii = 0
for epoch_data in epochs:
    mean_value = np.mean(epoch_data, axis=1)
    std_value = np.std(epoch_data, axis=1)
    energy_value = np.sum(np.square(epoch_data), axis=1)
    power_value = np.mean(np.square(epoch_data), axis=1)
    epoch_features = np.hstack([std_value, mean_value, energy_value, power_value, mean_mu[ii], mean_beta[ii]])
    feature_matrix.append(epoch_features)
    ii += 1

feature_matrix = np.array(feature_matrix)

In [179]:
print(feature_matrix.shape)
# 192 = 64 channels * 3 features (mean, power, energy) + 2 features (mu, beta)

(30, 258)


In [180]:
# normalize feature_matrix with min-max normalization
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
feature_matrix = min_max_scaler.fit_transform(feature_matrix)
min(feature_matrix[0]), max(feature_matrix[0])

(0.0, 0.7507587602582318)

In [181]:
# Example labels (replace with your actual labels)
labels = epochs.events[:, -1] - 1


In [182]:
np.unique(labels)

array([0, 1])

In [183]:
labels.shape, feature_matrix.shape

((30,), (30, 258))

In [184]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labels, test_size=0.2, random_state=42)


In [185]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Create and train the Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Predict labels on the test set
nb_predictions = nb_classifier.predict(X_test)

# Calculate accuracy
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')


Naive Bayes Accuracy: 33.33%


In [194]:
from sklearn.neural_network import MLPClassifier

# Create and train the ANN classifier
ann_classifier = MLPClassifier(hidden_layer_sizes=(5), max_iter=5800, activation='relu', random_state=42)
ann_classifier.fit(X_train, y_train)

# Predict labels on the test set
ann_predictions = ann_classifier.predict(X_test)

# Calculate accuracy
ann_accuracy = accuracy_score(y_test, ann_predictions)
print(f'ANN Accuracy: {ann_accuracy * 100:.2f}%')


ANN Accuracy: 50.00%


In [205]:
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

models = [
    ('Gradient Boosting', GradientBoostingClassifier(), {'model__n_estimators': [50, 100]}),
    ('Linear discriminant analysis', LinearDiscriminantAnalysis(), {'model__solver': ['svd'], 'model__tol': [0.0001, 0.00001]}),   
    ('SVM', SVC(), {'model__C': [0.5, 1, 3, 5, 10], 'model__gamma': [3, 4, 5, 10], 'model__degree': [1, 3, 5, 8], 'model__kernel': ['linear']}),
    ('KNN', KNeighborsClassifier(), {'model__n_neighbors': [4,5,6]}),
    ('MLP', MLPClassifier(), {'model__hidden_layer_sizes': [(100, 50), (200, 100)],  'model__max_iter': [5000]}),
    ('Decision Tree', DecisionTreeClassifier(), {'model__max_depth': [50, 100]}),
]

pipelines = []
# csp = CSP()
for name, model, param_grid in models:
    pipeline = Pipeline([
        # ('csp', csp),
        ('model', model)
    ])
    # param_grid['csp__n_components'] = [5, 10, 15]
    pipelines.append((name, pipeline, param_grid))


results = []
for name, pipeline, param_grid in pipelines:
    grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=shuffle_split, n_jobs=-1)
    grid_search.fit(feature_matrix, labels)
    results.append((name, grid_search))


res_grid = []
for name, grid_search in results:
    print(f"Model: {name}")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Cross-Validated Accuracy: {grid_search.best_score_:.2f}")
    print("\n")

/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/owalid/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value N

Model: Gradient Boosting
Best Parameters: {'model__n_estimators': 50}
Best Cross-Validated Accuracy: 0.45


Model: Linear discriminant analysis
Best Parameters: {'model__solver': 'svd', 'model__tol': 0.0001}
Best Cross-Validated Accuracy: 0.57


Model: SVM
Best Parameters: {'model__C': 3, 'model__degree': 1, 'model__gamma': 3, 'model__kernel': 'linear'}
Best Cross-Validated Accuracy: 0.63


Model: KNN
Best Parameters: {'model__n_neighbors': 4}
Best Cross-Validated Accuracy: 0.48


Model: MLP
Best Parameters: {'model__hidden_layer_sizes': (100, 50), 'model__max_iter': 5000}
Best Cross-Validated Accuracy: 0.55


Model: Decision Tree
Best Parameters: {'model__max_depth': 100}
Best Cross-Validated Accuracy: 0.55


